In [1]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd
import numpy as np
import pandas as pd
from ast import literal_eval
from pathlib import Path
from pprint import pprint
from sympy.combinatorics import Permutation

puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv'
my_submission_path = 'submission.csv'

puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)
my_submission_df = pd.read_csv(my_submission_path)

In [2]:
puzzles_df[puzzles_df["puzzle_type"] == 'globe_3/33']
# Two types: permutation & colors

,id,puzzle_type,solution_state,initial_state,num_wildcards
388,388,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,G;Q;G;D;E;T;E;N;U;H;H;U;O;F;T;B;S;M;A;N;D;V;V;...,6
389,389,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,H;A;P;K;G;N;U;G;D;V;E;L;D;O;E;Q;D;A;A;H;L;J;A;...,0
390,390,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,F;O;N;B;B;D;V;E;B;J;F;L;S;G;U;E;O;P;U;F;R;R;J;...,0
391,391,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N237;N257;N199;N215;N54;N261;N50;N6;N57;N32;N2...,0
392,392,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,L;A;O;U;T;Q;F;P;B;V;H;T;O;C;J;P;V;P;N;C;T;V;G;...,0
393,393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0


In [3]:
M = 66
globe = np.array(puzzles_df['initial_state'][392].split(';'))
globe = np.concatenate((globe[:M], globe[-M:]))
solution_globe = np.array(puzzles_df['solution_state'][392].split(';'))
solution_globe = np.concatenate((solution_globe[:M], solution_globe[-M:]))

In [4]:
idx = 343

N = 1+1
M = int(2*33)

# Solving 2xM

In [5]:
def print_globe(b):
    print(np.asarray(b).reshape(2, M))

apply_move = {}

def set_moves(verbose = False): # Assumes M is reset
    test_globe = np.array([i for i in range(2*M)])
    r0_perm = np.array([i for i in range(2*M)])
    r1_perm = np.array([i for i in range(2*M)])
    f0_perm = np.array([i for i in range(2*M)])

    r0_perm[M-1] = 0
    for i in range(0, M-1):
        r0_perm[i] = i+1

    r1_perm[2 * M - 1] = M
    for i in range(M, 2*M-1):
        r1_perm[i] = i+1

    for i in range(M//2):
        f0_perm[i] = M + M//2 - 1 - i

    for i in range(M, M + M//2):
        f0_perm[i] = M//2-1 - (i-M)

    nr0_perm = np.array([i for i in range(2*M)])
    nr1_perm = np.array([i for i in range(2*M)])

    for i in range(M-1):
        nr0_perm = nr0_perm[r0_perm]

    for i in range(M-1):
        nr1_perm = nr1_perm[r1_perm]

    def r0(arr, moves):
        moves.append('r0')
        arr = arr[r0_perm]
        return arr

    def nr0(arr, moves):
        moves.append('-r0')
        arr = arr[nr0_perm]
        return arr

    def r1(arr, moves):
        moves.append('r1')
        arr = arr[r1_perm]
        return arr

    def nr1(arr, moves):
        moves.append('-r1')
        arr = arr[nr1_perm]
        return arr

    def f0(arr, moves):
        moves.append('f0')
        arr = arr[f0_perm]
        return arr

    apply_move['r0'] = r0
    apply_move['-r0'] = nr0
    apply_move['r1'] = r1
    apply_move['-r1'] = nr1
    apply_move['f0'] = f0

    def create_f_function(i, cur_perm):
        return lambda arr, moves: (moves.append('f' + str(i)), arr[cur_perm])[1]

    for i in range(1, M):
        cur_perm = np.array([i for i in range(2*M)])

        for j in range(i):
            cur_perm = cur_perm[r0_perm]
            cur_perm = cur_perm[r1_perm]

        cur_perm = cur_perm[f0_perm]

        for j in range(i):
            cur_perm = cur_perm[nr0_perm]
            cur_perm = cur_perm[nr1_perm]

        apply_move['f' + str(i)] = create_f_function(i, cur_perm)

    if verbose:
        print("r0")
        print_globe(r0(test_globe, []))
        print("nr0")
        print_globe(nr0(test_globe, []))
        print("r1")
        print_globe(r1(test_globe, []))
        print("f0")
        print_globe(f0(test_globe, []))
        print_globe(apply_move['f1'](test_globe, []))
        print_globe(apply_move['f2'](test_globe, []))

    set_half_turn_moves(verbose)

def set_half_turn_moves(verbose):
    test_globe = np.array([i for i in range(2*M)])
    def half_turn_ccw(arr, moves):
        for i in range(M//2):
            arr = apply_move['-r0'](arr, moves)

        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)

        for i in range(M//2):
            arr = apply_move['r0'](arr, moves)
        return arr

    def half_turn_cw(arr, moves):
        for i in range(M//2):
            arr = apply_move['-r0'](arr, moves)

        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)

        for i in range(M//2):
            arr = apply_move['r0'](arr, moves)
        return arr

    apply_move['half_turn'] = half_turn_ccw
    apply_move['rev_half_turn'] = half_turn_cw

    def top_right_cw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr

    def top_right_ccw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r0'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr

    def top_left_ccw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['-r1'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr

    def top_left_cw(arr, moves):
        arr = apply_move['f0'](arr, moves)
        arr = apply_move['r1'](arr, moves)
        arr = apply_move['f0'](arr, moves)
        return arr

    apply_move['top_right_cw'] = top_right_cw
    apply_move['top_right_ccw'] = top_right_ccw
    apply_move['top_left_cw'] = top_left_cw
    apply_move['top_left_ccw'] = top_left_ccw

    if verbose:
        print_globe(half_turn_ccw(test_globe, []))
        print_globe(half_turn_cw(test_globe, []))
        print_globe(top_right_cw(test_globe, []))
        print_globe(top_right_ccw(test_globe, []))
        print_globe(top_left_cw(test_globe, []))
        print_globe(top_left_ccw(test_globe, []))

set_moves(True)

r0
[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
   19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
   37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
   55  56  57  58  59  60  61  62  63  64  65   0]
 [ 66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
   84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
  102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
  120 121 122 123 124 125 126 127 128 129 130 131]]
nr0
[[ 65   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
   17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
   35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
   53  54  55  56  57  58  59  60  61  62  63  64]
 [ 66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
   84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
  102 103 104 105 106 107

In [6]:
def set_left_half(globe: np.array, solution_globe: np.array, moves: list) -> np.array:
    """
    MOVES CAN STILL BE OPTIMIZED. Optimize bottom left first.
    Note we can directly model some operations as a shortest path problem instead of constructing by hand.

    Sets the left half of the globe to anything (the right half is arbitrary after).
    Returns it (original unaffected).
    'moves' changed in place.

    We first set the top left then the bottom left, both from left to right.
    Most of the puzzles have duplicates, and this function accounts for that to decrease moves.

    TOP LEFT:
        For the top left, say we want to set the ith element.
        Then, our goal is to use any element with value ai (remember there are duplicates).
        Let's get it in the bottom row at position i+m/2-1 so we can apply fi to finish.

        Consider each candidate element and pick the one that takes the least moves.

        If it is in the top, try all available f's. Now it is in the bottom.

        Now the element must be in the bottom, say at position j. The following algo is called fastest_rotation() in the code:
        It will take j-(i+m/2-1) rotations (if negative, use inverse).
            POSSIBLE OPTIMIZATION: we could use 2 f moves to move the element faster. Use BFS for optimal.

        Finally, Apply fi to set the element.

    BOTTOM LEFT:
        For the bottom left, say we want to set the ith element.
            Note that i > 0, i=0 handled separately similar to TOP LEFT.
        An invariant is that the elements 0, 1, ... i-1 are contiguous and i-1 is at M+M//2-1 in the bottom (center).

        Consider each candidate element (any with value ai) and pick the one that takes the least moves.

        If it is in the top, use f to put it in the bottom, now it is in the right half.
        If it is in the bottom and in the left half, rotate until it is in the right half.

        Now the element must be in the right half. The following algo is called merge_i() in the code:
        Use r1 until element 0 is at index M or element i is at M + M//2, whichever occurs sooner.
        Then, use top_right_ccw until element i-1 is at M+M//2-1.
        Repeat until element i-1 is adjacent to element i (at M + M//2). Then finish with r1.

    NUMBER OF MOVES:
        Empirical results for 3/33:
            N0:N1... (no duplicates)
                Top left ~390
                Bot left ~1550

            (AAAAAABBBBBBC...) (6 original duplicates)
                Top left ~180
                Bot left ~800

        Theory:
            Let x be the number of duplicates.

            Top left:
                ~M/2(x+1) moves per element.
                    If x=1, the element is about M/4 away from i+m/2-1. We generalize this as x+1 points on a circle.
            Bottom left:
                ~2*M/(x+1)
                    Using the x+1 points on a circle argument, bottom point is M/(x+1) away.
                    To decrease dist by 1, we'll need r1 + 1 top top_right_ccw.
                    So worse case is 4 moves per 1 dist. However, for small i we can do multiple top_right_ccw at once.

    OPTIMAL PERM TO SET:
        WLOG consider setting the top left. It is optimal to set it to 0, 1, ... M/4-1, 3M/4, 2, ... M-1

        Move_top_bot_helper() will insert the remaining elements (M/4 ... 3M/4-1) randomly.

        Note that an array of length X has X^2/4 inversions on average.
        Thus, we'll have (M/2)^2/4 inversions from these remaining elements regardless of our choice of top left.

        Let's calculate the # of inversions between the remaining elements and those in the top left.
        It suffices to calculate it for a single element since they all have the same relative value with respect to top left.
        There are M//2+1 "locations" it could equally likely be (before 1st top left, between 1st, 2nd, ...).
        The inversions respectively are M/4, M/4-1, ... 0 ... M/4.
        Thus, the average # of inversions for each of remaining element with elements in top left = M/8.
        We can prove the optimality via:
            Inversions contributed by each element are independent.
            Minimum expected inversions achieved at median via calculus: minimize (k)(k-1)/2 + (m-k)(m-k-1)/2
        This yields M/8 * M/2 more inversions.

        The total is M^2/8 inversions, which is half the number if top was just randomly permuted (M^2/4).
    """

    # Optimal top left and bottom left
    sorted_top = list(solution_globe[:M])
    sorted_bot = list(solution_globe[M:])
    top_left = sorted_top[:M//4] + sorted_top[-(M//2 - M//4):]
    bot_left = sorted_bot[:M//4] + sorted_bot[-(M//2 - M//4):]

    def idx_after_f(x, i):
        # Index of i (i is in top) after using fx (gets mirrored and placed at bottom)
        return M + x + (M//2-1 - (i-x))

    def fastest_rotation(j, k):
        # Returns the shortest rotation moves to place j at k on bottom
        dist_r1 = (j-k) % M
        dist_nr1 = (k-j) % M
        if dist_r1 < dist_nr1:
            # Faster to use r1
            return ['r1'] * dist_r1
        else:
            # Faster to go in the other direction and use -r1
            return ['-r1'] * dist_nr1

    # Set top left element by element
    for i in range(M//2):
        if globe[i] == top_left[i]:
            # Lucky! It's already set.
            continue

        best_moves = None
        candidates = []  # (moves, index of candidate in bottom) after reducing to in bottom case

        # Top
        for j in range(i+1, M):
            if globe[j] == top_left[i]:
                # Try all possible f{k} moves
                for x in range(max(i+1, j-M//2+1), min(j, M//2)+1):
                    candidates.append(([f'f{x}'], idx_after_f(x, j)))

        # Bottom
        for j in range(M, 2*M):
            if globe[j] == top_left[i]:
                candidates.append(([], j))

        # Now we've reduces to bottom case
        # Find candidate which takes the least moves
        final_idx_bot = M + i+M//2-1  # Index we want the element before applying fi
        for cur_moves, bot_j in candidates:
            cur_moves += fastest_rotation(bot_j, final_idx_bot)

            # Update if better
            if best_moves is None or len(cur_moves) < len(best_moves):
                best_moves = cur_moves

        # Apply final fi
        best_moves.append(f'f{i}')

        # Apply moves to the globe
        for move in best_moves:
            globe = apply_move[move](globe, moves)

    orig_moves = len(moves)
    print(orig_moves)


    # Set bottom left element by element

    def merge_i(idx_0, idx_i):
        # Returns the moves to make elements i-1 and i adjacent (i is in the bottom left)

        moves = []

        while idx_i - idx_0 > i:
            # Use r1 until idx_0 is at M or idx_i is at M + M//2
            max_r1 = min(idx_0 - M, idx_i - (M+M//2))
            moves.extend(['r1'] * max_r1)
            idx_i -= max_r1
            idx_0 -= max_r1

            # Use top_right_ccw until i-1 is at center (M + M//2-1)
            idx_i_minus_one = idx_0 + i-1
            amt_rotate = (M + M//2-1) - idx_i_minus_one
            moves.extend([f'f{M//2}'] + ['-r1']*amt_rotate + [f'f{M//2}'])
            idx_0 += amt_rotate

        return moves

    for i in range(M//2):
        best_moves = None

        if i == 0:
            # Handle element 0 separately, very similarly to how we did top left.
            candidates = []  # (moves, index of element) after reducing to in bottom case.

            # At top
            for j in range(M//2, M):
                if globe[j] == bot_left[i]:
                    candidates.append(([f'f{M//2}'], idx_after_f(M//2, j)))

            # At bot
            for j in range(M, 2*M):
                if globe[j] == bot_left[i]:
                    candidates.append(([], j))

            # Now we've reduced to bottom case
            # Find best candidate
            for cur_moves, idx_candidate in candidates:
                cur_moves += fastest_rotation(idx_candidate, M + M//2-1)

                # Update if better
                if best_moves is None or len(cur_moves) < len(best_moves):
                    best_moves = cur_moves
        else:
            candidates = [] # (moves, index of 0, index of candidate in bottom) after reducing to in bottom right case.
            idx_0 = M + M//2-i

            # Top
            for j in range(M//2, M):
                if globe[j] == bot_left[i]:
                    # Apply f{M//2}
                    candidates.append(([f'f{M//2}'], idx_0, idx_after_f(M//2, j)))

            # Bot left
            for j in range(M, idx_0):
                if globe[j] == bot_left[i]:
                    # Use r1 until candidate is in bot right
                    num_r1 = j-M+1
                    candidates.append((['r1'] * num_r1, idx_0 - num_r1, 2*M-1))

            # Bot right
            for j in range(M + M//2, 2*M):
                if globe[j] == bot_left[i]:
                    candidates.append(([], idx_0, j))

            # Now we've reduced to bottom right case
            # Find best candidate
            for cur_moves, idx_0, idx_candidate in candidates:
                cur_moves += merge_i(idx_0, idx_candidate)

                # Update if better
                if best_moves is None or len(cur_moves) < len(best_moves):
                    best_moves = cur_moves

            # Apply r1 to now have element i at center (M + M//2-1)
            best_moves.append('r1')

        # Apply moves to the globe
        for move in best_moves:
            globe = apply_move[move](globe, moves)

    print(len(moves) - orig_moves)

    assert (globe[:M//2] == top_left).all()
    assert (globe[M:M+M//2] == bot_left).all()

    return globe

moves = []
# g_after = set_left_half(globe, solution_globe, moves)
print_globe(set_left_half(globe, solution_globe, moves))
print_globe(solution_globe)

208
823
[['A' 'A' 'A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'Q' 'Q'
  'Q' 'Q' 'Q' 'S' 'S' 'S' 'S' 'S' 'S' 'U' 'U' 'U' 'U' 'U' 'U' 'N' 'J' 'H'
  'K' 'I' 'O' 'N' 'K' 'O' 'H' 'K' 'P' 'K' 'F' 'O' 'H' 'E' 'J' 'P' 'H' 'H'
  'J' 'L' 'E' 'N' 'R' 'P' 'M' 'P' 'G' 'N' 'J' 'I']
 ['B' 'B' 'B' 'B' 'B' 'B' 'D' 'D' 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'R' 'R'
  'R' 'R' 'R' 'T' 'T' 'T' 'T' 'T' 'T' 'V' 'V' 'V' 'V' 'V' 'V' 'O' 'J' 'G'
  'F' 'I' 'I' 'M' 'G' 'I' 'N' 'L' 'L' 'N' 'K' 'P' 'O' 'I' 'G' 'J' 'P' 'O'
  'G' 'M' 'L' 'M' 'L' 'Q' 'G' 'M' 'M' 'L' 'H' 'K']]
[['A' 'A' 'A' 'A' 'A' 'A' 'C' 'C' 'C' 'C' 'C' 'C' 'E' 'E' 'E' 'E' 'E' 'E'
  'G' 'G' 'G' 'G' 'G' 'G' 'I' 'I' 'I' 'I' 'I' 'I' 'K' 'K' 'K' 'K' 'K' 'K'
  'M' 'M' 'M' 'M' 'M' 'M' 'O' 'O' 'O' 'O' 'O' 'O' 'Q' 'Q' 'Q' 'Q' 'Q' 'Q'
  'S' 'S' 'S' 'S' 'S' 'S' 'U' 'U' 'U' 'U' 'U' 'U']
 ['B' 'B' 'B' 'B' 'B' 'B' 'D' 'D' 'D' 'D' 'D' 'D' 'F' 'F' 'F' 'F' 'F' 'F'
  'H' 'H' 'H' 'H' 'H' 'H' 'J' 'J' 'J' 'J' 'J' 'J' 'L' 'L' 'L' 'L' 'L' 'L'
  'N' 'N' 'N' 'N' 'N' 'N

In [7]:
def min_inversion_ordering_array(arr: np.array) -> np.array:
    """
    globe is a 2xm globe.

    When globe has duplicates, there are many sorted "orderings" when treating each element as distinct.

    For example:
    AABBCC is the original array.
    Then there are 2^3 sorted orderings e.g. 012345, 102345, 103245 ...

    This function returns the ordering that has minimum inversions.
    The optimal ordering has elements of the same value mapped to the
    corresponding increasing subsequence. Proof via exchange argument.

    For example (one layer only):
    BACCAB => 204513
    """

    res = np.ones(M, dtype=np.int8)
    idx = {}  # idx[value] = next index for 'value'

    # Get the first index in the sorted array
    for i, x in enumerate(sorted(arr)):
        if x not in idx:
            idx[x] = i

    # Find the ordering
    for i in range(M):
        res[i] = idx[arr[i]]
        idx[arr[i]] += 1

    return res

In [8]:
def min_inversion_ordering(globe: np.array) -> np.array:
    """
    Returns the ordering of elements that minimizes inversion in the
    bottom and top layer. See min_inversion_ordering_array for more details.
    """

    top = globe[:M]
    bot = globe[M:]

    return np.concatenate((min_inversion_ordering_array(top),
                           min_inversion_ordering_array(bot) + M))